In [1]:
import os
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD

import gensim
from gensim.models.keyedvectors import KeyedVectors

C:\Users\PC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\PC\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 512
EPOCHS = 2000
LATENT_DIM = 50

In [3]:
# load in the data
input_texts = []
target_texts = []
for line in open('C://Users//PC//Desktop//Advanced NLP//Data//Poetry//robert_frost.txt'):
    line = line.rstrip()
    
    if not line:
        continue
    
    input_line = '<sos> ' + line
    output_line = line + " <eos>"
    
    input_texts.append(input_line)
    target_texts.append(output_line)

In [4]:
input_texts[0]

'<sos> Two roads diverged in a yellow wood,'

In [5]:
target_texts[0]

'Two roads diverged in a yellow wood, <eos>'

In [6]:
# how many data for training and validation
len(target_texts)

1436

In [7]:
all_lines = input_texts + target_texts

In [8]:
## TOKENIZATION ##
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

In [9]:
input_sequences

[[1, 104, 537, 538, 9, 7, 539, 540],
 [1, 5, 541, 6, 65, 31, 934, 141],
 [1, 5, 27, 24, 935, 152, 6, 221],
 [1, 5, 167, 67, 24, 17, 128, 17, 6, 65],
 [1, 4, 40, 11, 936, 9, 3, 937],
 [1, 91, 189, 3, 542, 17, 77, 17, 938],
 [1, 5, 142, 265, 3, 222, 939],
 [1, 319, 11, 14, 543, 5, 223, 940],
 [1, 122, 17, 15, 13, 3, 544, 55],
 [1, 23, 545, 52, 404, 111, 3, 546],
 [1, 5, 141, 13, 941, 942, 405],
 [1, 9, 943, 42, 547, 23, 944, 548],
 [1, 549, 6, 266, 3, 224, 15, 267, 945],
 [1, 153, 946, 92, 81, 947, 18, 4, 81],
 [1, 6, 948, 28, 6, 123, 154, 78, 268],
 [1, 6, 190, 27, 949, 53, 19, 7, 950],
 [1, 550, 551, 5, 551, 951],
 [1, 104, 537, 538, 9, 7, 540, 5, 952],
 [1, 6, 189, 3, 24, 406, 953, 954],
 [1, 5, 13, 93, 82, 21, 3, 955],
 [1, 225, 226, 320, 59, 6, 72, 6, 269],
 [1, 26, 96, 32, 9, 3, 956, 957],
 [1, 12, 79, 31, 38, 25, 958, 68],
 [1, 4, 407, 26, 226, 959, 35, 19, 270],
 [1, 44, 143, 552, 73, 72, 11, 960],
 [1, 4, 321, 271, 7, 961, 227],
 [1, 228, 3, 226, 5, 962, 553],
 [1, 3, 963, 554, 

In [10]:
# find max seq length
max_sequence_length_from_data = max(len(s) for s in input_sequences)
print('Max sequence length:', max_sequence_length_from_data)

Max sequence length: 12


In [11]:
# get word -£ integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))
assert('<sos>' in word2idx)
assert('<eos>' in word2idx)

Found 3056 unique tokens.


In [12]:
# padding
max_sequence_length = min(max_sequence_length_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')
input_sequences.shape

(1436, 12)

In [13]:
input_sequences[0]

array([  1, 104, 537, 538,   9,   7, 539, 540,   0,   0,   0,   0])

In [14]:
# Load WordVectors
print("loading word2vec model…")
word2vec_model = KeyedVectors.load_word2vec_format('D:\\WordEmbeddings\\GoogleNews-vectors-negative300.bin', binary=True)

loading word2vec model…


C:\Users\PC\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
def getVector(str):
    if str in word2vec_model:
        return word2vec_model[str]
    else:
        return None
    
def isInModel(str):
    return str in word2vec_model

In [16]:
# create a weight matrix for words in training docs
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
print(num_words)
embedding_matrix = np.zeros((num_words, 300))

for word, i in word2idx.items():
    if i < num_words:
        embedding_vector = getVector(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros.
            embedding_matrix[i] = embedding_vector

3000


In [17]:
# one-hot the targets (can't use sparse cross-entropy)
one_hot_targets = np.zeros((len(input_sequences), max_sequence_length, num_words))
for i, target_sequence in enumerate(target_sequences):
    for t, word in enumerate(target_sequence):
        if word > 0:
            one_hot_targets[i, t, word] = 1

In [18]:
one_hot_targets

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [19]:
one_hot_targets.shape

(1436, 12, 3000)

In [20]:
# load pre-trained word embeddings into an Embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  # trainable=False
)

In [21]:
print('Building model...')

# create an LSTM network with a single LSTM
input_ = Input(shape=(max_sequence_length,))

initial_h = Input(shape=(LATENT_DIM,))
initial_c = Input(shape=(LATENT_DIM,))

x = embedding_layer(input_)
lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, _, _ = lstm(x, initial_state=[initial_h, initial_c]) # don't need the states here
dense = Dense(num_words, activation='softmax')
output = dense(x)

Building model...


In [22]:
model = Model([input_, initial_h, initial_c], output)
model.compile(
  loss='categorical_crossentropy',
  # optimizer='rmsprop',
  optimizer=Adam(lr=0.01),
  # optimizer=SGD(lr=0.01, momentum=0.9),
  metrics=['accuracy']
)

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 12, 300)      900000      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
lstm_1 (LS

In [24]:
print('Training model...')
z = np.zeros((len(input_sequences), LATENT_DIM))
r = model.fit(
  [input_sequences, z, z],
  one_hot_targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

Training model...
Train on 1148 samples, validate on 288 samples
Epoch 1/2000
1148/1148 [==============================] - 1s 1ms/step - loss: 5.6078 - acc: 0.0503 - val_loss: 5.5779 - val_acc: 0.0833
Epoch 2/2000
1148/1148 [==============================] - 0s 374us/step - loss: 5.4014 - acc: 0.0833 - val_loss: 5.1182 - val_acc: 0.0833
Epoch 3/2000
1148/1148 [==============================] - 0s 374us/step - loss: 4.8454 - acc: 0.0833 - val_loss: 4.8030 - val_acc: 0.0833
Epoch 4/2000
1148/1148 [==============================] - 0s 382us/step - loss: 4.4852 - acc: 0.0833 - val_loss: 4.8506 - val_acc: 0.0833
Epoch 5/2000
1148/1148 [==============================] - 0s 385us/step - loss: 4.4093 - acc: 0.0833 - val_loss: 5.0165 - val_acc: 0.0833
Epoch 6/2000
1148/1148 [==============================] - 0s 384us/step - loss: 4.4136 - acc: 0.0833 - val_loss: 5.0328 - val_acc: 0.0833
Epoch 7/2000
1148/1148 [==============================] - 0s 381us/step - loss: 4.3441 - acc: 0.0833 - val_lo

1148/1148 [==============================] - 0s 378us/step - loss: 3.1984 - acc: 0.1405 - val_loss: 5.0806 - val_acc: 0.0969
Epoch 60/2000
1148/1148 [==============================] - 0s 378us/step - loss: 3.1805 - acc: 0.1429 - val_loss: 5.0899 - val_acc: 0.0964
Epoch 61/2000
1148/1148 [==============================] - 0s 378us/step - loss: 3.1619 - acc: 0.1468 - val_loss: 5.0972 - val_acc: 0.0972
Epoch 62/2000
1148/1148 [==============================] - 0s 375us/step - loss: 3.1440 - acc: 0.1490 - val_loss: 5.1019 - val_acc: 0.0984
Epoch 63/2000
1148/1148 [==============================] - 0s 378us/step - loss: 3.1281 - acc: 0.1498 - val_loss: 5.1058 - val_acc: 0.0972
Epoch 64/2000
1148/1148 [==============================] - 0s 378us/step - loss: 3.1103 - acc: 0.1528 - val_loss: 5.1113 - val_acc: 0.0978
Epoch 65/2000
1148/1148 [==============================] - 0s 378us/step - loss: 3.0930 - acc: 0.1519 - val_loss: 5.1177 - val_acc: 0.0966
Epoch 66/2000
1148/1148 [================

Epoch 118/2000
1148/1148 [==============================] - 0s 378us/step - loss: 2.4013 - acc: 0.2413 - val_loss: 5.3726 - val_acc: 0.0897
Epoch 119/2000
1148/1148 [==============================] - 0s 382us/step - loss: 2.3905 - acc: 0.2419 - val_loss: 5.3713 - val_acc: 0.0914
Epoch 120/2000
1148/1148 [==============================] - 0s 380us/step - loss: 2.3800 - acc: 0.2428 - val_loss: 5.3717 - val_acc: 0.0906
Epoch 121/2000
1148/1148 [==============================] - 0s 383us/step - loss: 2.3702 - acc: 0.2448 - val_loss: 5.3775 - val_acc: 0.0897
Epoch 122/2000
1148/1148 [==============================] - 0s 380us/step - loss: 2.3605 - acc: 0.2464 - val_loss: 5.3830 - val_acc: 0.0883
Epoch 123/2000
1148/1148 [==============================] - 0s 384us/step - loss: 2.3507 - acc: 0.2479 - val_loss: 5.3831 - val_acc: 0.0891
Epoch 124/2000
1148/1148 [==============================] - 0s 378us/step - loss: 2.3407 - acc: 0.2506 - val_loss: 5.3850 - val_acc: 0.0894
Epoch 125/2000
1148/

1148/1148 [==============================] - 0s 379us/step - loss: 1.9379 - acc: 0.2990 - val_loss: 5.6049 - val_acc: 0.0868
Epoch 177/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.9317 - acc: 0.3007 - val_loss: 5.6087 - val_acc: 0.0859
Epoch 178/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.9260 - acc: 0.3028 - val_loss: 5.6054 - val_acc: 0.0868
Epoch 179/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.9186 - acc: 0.3033 - val_loss: 5.6090 - val_acc: 0.0851
Epoch 180/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.9122 - acc: 0.3036 - val_loss: 5.6153 - val_acc: 0.0845
Epoch 181/2000
1148/1148 [==============================] - 0s 380us/step - loss: 1.9058 - acc: 0.3045 - val_loss: 5.6182 - val_acc: 0.0862
Epoch 182/2000
1148/1148 [==============================] - 0s 381us/step - loss: 1.9000 - acc: 0.3068 - val_loss: 5.6249 - val_acc: 0.0862
Epoch 183/2000
1148/1148 [=========

1148/1148 [==============================] - 0s 377us/step - loss: 1.6044 - acc: 0.3590 - val_loss: 5.8344 - val_acc: 0.0842
Epoch 235/2000
1148/1148 [==============================] - 0s 379us/step - loss: 1.5993 - acc: 0.3586 - val_loss: 5.8331 - val_acc: 0.0810
Epoch 236/2000
1148/1148 [==============================] - 0s 379us/step - loss: 1.5933 - acc: 0.3603 - val_loss: 5.8321 - val_acc: 0.0813
Epoch 237/2000
1148/1148 [==============================] - 0s 376us/step - loss: 1.5883 - acc: 0.3604 - val_loss: 5.8380 - val_acc: 0.0819
Epoch 238/2000
1148/1148 [==============================] - 0s 376us/step - loss: 1.5828 - acc: 0.3613 - val_loss: 5.8426 - val_acc: 0.0828
Epoch 239/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.5780 - acc: 0.3650 - val_loss: 5.8545 - val_acc: 0.0842
Epoch 240/2000
1148/1148 [==============================] - 0s 378us/step - loss: 1.5725 - acc: 0.3653 - val_loss: 5.8567 - val_acc: 0.0828
Epoch 241/2000
1148/1148 [=========

...

1148/1148 [==============================] - 0s 380us/step - loss: 0.6163 - acc: 0.5613 - val_loss: 7.9255 - val_acc: 0.0671
Epoch 1685/2000
1148/1148 [==============================] - 0s 383us/step - loss: 0.6161 - acc: 0.5609 - val_loss: 7.9249 - val_acc: 0.0666
Epoch 1686/2000
1148/1148 [==============================] - 0s 380us/step - loss: 0.6157 - acc: 0.5607 - val_loss: 7.9212 - val_acc: 0.0666
Epoch 1687/2000
1148/1148 [==============================] - 0s 385us/step - loss: 0.6154 - acc: 0.5603 - val_loss: 7.9202 - val_acc: 0.0663
Epoch 1688/2000
1148/1148 [==============================] - 0s 382us/step - loss: 0.6148 - acc: 0.5608 - val_loss: 7.9168 - val_acc: 0.0657
Epoch 1689/2000
1148/1148 [==============================] - 0s 383us/step - loss: 0.6146 - acc: 0.5602 - val_loss: 7.9132 - val_acc: 0.0654
Epoch 1690/2000
1148/1148 [==============================] - 0s 392us/step - loss: 0.6147 - acc: 0.5592 - val_loss: 7.9151 - val_acc: 0.0686
Epoch 1691/2000
1148/1148 [==

1148/1148 [==============================] - 0s 382us/step - loss: 0.6137 - acc: 0.5605 - val_loss: 7.9460 - val_acc: 0.0657
Epoch 1743/2000
1148/1148 [==============================] - 0s 386us/step - loss: 0.6136 - acc: 0.5584 - val_loss: 7.9392 - val_acc: 0.0654
Epoch 1744/2000
1148/1148 [==============================] - 0s 390us/step - loss: 0.6134 - acc: 0.5606 - val_loss: 7.9434 - val_acc: 0.0645
Epoch 1745/2000
1148/1148 [==============================] - 0s 383us/step - loss: 0.6129 - acc: 0.5608 - val_loss: 7.9566 - val_acc: 0.0657
Epoch 1746/2000
1148/1148 [==============================] - 0s 383us/step - loss: 0.6130 - acc: 0.5607 - val_loss: 7.9620 - val_acc: 0.0660
Epoch 1747/2000
1148/1148 [==============================] - 0s 380us/step - loss: 0.6124 - acc: 0.5614 - val_loss: 7.9597 - val_acc: 0.0657
Epoch 1748/2000
1148/1148 [==============================] - 0s 378us/step - loss: 0.6131 - acc: 0.5608 - val_loss: 7.9567 - val_acc: 0.0660
Epoch 1749/2000
1148/1148 [==

1148/1148 [==============================] - 0s 380us/step - loss: 0.6119 - acc: 0.5609 - val_loss: 7.9737 - val_acc: 0.0631
Epoch 1801/2000
1148/1148 [==============================] - 0s 382us/step - loss: 0.6123 - acc: 0.5611 - val_loss: 7.9684 - val_acc: 0.0628
Epoch 1802/2000
1148/1148 [==============================] - 0s 381us/step - loss: 0.6115 - acc: 0.5612 - val_loss: 7.9673 - val_acc: 0.0625
Epoch 1803/2000
1148/1148 [==============================] - 0s 382us/step - loss: 0.6119 - acc: 0.5608 - val_loss: 7.9738 - val_acc: 0.0637
Epoch 1804/2000
1148/1148 [==============================] - 0s 382us/step - loss: 0.6129 - acc: 0.5608 - val_loss: 7.9843 - val_acc: 0.0642
Epoch 1805/2000
1148/1148 [==============================] - 0s 378us/step - loss: 0.6129 - acc: 0.5600 - val_loss: 7.9860 - val_acc: 0.0625
Epoch 1806/2000
1148/1148 [==============================] - 0s 387us/step - loss: 0.6120 - acc: 0.5608 - val_loss: 7.9792 - val_acc: 0.0631
Epoch 1807/2000
1148/1148 [==

1148/1148 [==============================] - 0s 382us/step - loss: 0.6183 - acc: 0.5604 - val_loss: 7.9746 - val_acc: 0.0666
Epoch 1859/2000
1148/1148 [==============================] - 0s 380us/step - loss: 0.6182 - acc: 0.5605 - val_loss: 7.9726 - val_acc: 0.0683
Epoch 1860/2000
1148/1148 [==============================] - 0s 380us/step - loss: 0.6175 - acc: 0.5600 - val_loss: 7.9799 - val_acc: 0.0700
Epoch 1861/2000
1148/1148 [==============================] - 0s 381us/step - loss: 0.6167 - acc: 0.5602 - val_loss: 7.9775 - val_acc: 0.0718
Epoch 1862/2000
1148/1148 [==============================] - 0s 379us/step - loss: 0.6160 - acc: 0.5602 - val_loss: 7.9822 - val_acc: 0.0712
Epoch 1863/2000
1148/1148 [==============================] - 0s 384us/step - loss: 0.6154 - acc: 0.5604 - val_loss: 7.9877 - val_acc: 0.0692
Epoch 1864/2000
1148/1148 [==============================] - 0s 378us/step - loss: 0.6162 - acc: 0.5602 - val_loss: 7.9897 - val_acc: 0.0680
Epoch 1865/2000
1148/1148 [==

KeyboardInterrupt: 

In [ ]:
# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
# accuracies
plt.plot(r.history['acc'], label='acc')
plt.plot(r.history['val_acc'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
# make a sampling model
input2 = Input(shape=(1,)) # we'll only input one word at a time
x = embedding_layer(input2)
x, h, c = lstm(x, initial_state=[initial_h, initial_c]) # now we need states to feed back in
output2 = dense(x)
sampling_model = Model([input2, initial_h, initial_c], [output2, h, c])

In [ ]:
# reverse word2idx dictionary to get back words
# during prediction
idx2word = {v:k for k, v in word2idx.items()}

In [ ]:
def sample_line():
    # initial inputs
    np_input = np.array([[ word2idx['<sos>'] ]])
    h = np.zeros((1, LATENT_DIM))
    c = np.zeros((1, LATENT_DIM))

    # so we know when to quit
    eos = word2idx['<eos>']

    # store the output here
    output_sentence = []

    for _ in range(max_sequence_length):
        o, h, c = sampling_model.predict([np_input, h, c])

        # print("o.shape:", o.shape, o[0,0,:10])
        # idx = np.argmax(o[0,0])
        probs = o[0,0]
        if np.argmax(probs) == 0:
            print("wtf")

        probs[0] = 0
        probs /= probs.sum()
        idx = np.random.choice(len(probs), p=probs)

        if idx == eos:
            break

    # accumulate output
    output_sentence.append(idx2word.get(idx, '<WTF %s>' % idx))

    # make the next input into model
    np_input[0,0] = idx

    return ' '.join(output_sentence)

In [ ]:
# generate a 4 line poem
while True:
    for _ in range(4):
        print(sample_line())
    
    ans = input("---generate another? [Y/n]---")
    
    if ans and ans[0].lower().startswith('n'):
        break
